# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# 因為需要把類別型與數值型特徵都加入, 故使用最簡版的特徵工程
LEncoder = LabelEncoder()
MMEncoder = MinMaxScaler()
for c in df.columns:
    df[c] = df[c].fillna(-1)
    if df[c].dtype == 'object':
        df[c] = LEncoder.fit_transform(list(df[c].values))
    df[c] = MMEncoder.fit_transform(df[c].values.reshape(-1, 1).astype('float'))
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1.0,0.121348,1.0,0.283951,0.125,0.0,0.769118,0.014151,0.000000,1.000000
1,0.0,0.213483,0.0,0.481481,0.125,0.0,0.876471,0.139136,0.557823,0.333333
2,1.0,0.396629,0.0,0.333333,0.000,0.0,0.983824,0.015469,0.000000,1.000000
3,0.0,0.305618,0.0,0.444444,0.125,0.0,0.072059,0.103644,0.380952,1.000000
4,1.0,0.016854,1.0,0.444444,0.000,0.0,0.694118,0.015713,0.000000,1.000000


In [3]:
# 梯度提升樹擬合後, 將結果依照重要性由高到低排序 (note : D27作業中'Ticket'是第一名特徵, 'Age'是數值特徵中排名最高者)
estimator = GradientBoostingClassifier()
estimator.fit(df.values, train_Y)
feats = pd.Series(data=estimator.feature_importances_, index=df.columns)
feats = feats.sort_values(ascending=False)
feats

Sex         0.444711
Pclass      0.111737
Ticket      0.097623
Fare        0.086930
Name        0.081984
Age         0.073800
Cabin       0.071703
Embarked    0.012220
Parch       0.011564
SibSp       0.007727
dtype: float64

In [4]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""

df['Sex'] = df['Sex'].fillna('None')
df['Age'] = df['Age'].fillna(df['Age'].mean())

mean_df = df.groupby(['Sex'])['Age'].mean().reset_index()
mode_df = df.groupby(['Sex'])['Age'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Sex'])['Age'].median().reset_index()
max_df = df.groupby(['Sex'])['Age'].max().reset_index()
min_df = df.groupby(['Sex'])['Age'].min().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Sex'])
temp = pd.merge(temp, median_df, how='left', on=['Sex'])
temp = pd.merge(temp, max_df, how='left', on=['Sex'])
temp = pd.merge(temp, min_df, how='left', on=['Sex'])
temp.columns = ['Sex', 'Age_Mean', 'Age_Mode', 'Age_Median', 'Age_Max', 'Age_Min']
temp


,Sex,Age_Mean,Age_Mode,Age_Median,Age_Max,Age_Min
0,0.0,0.296729,0.0,0.308642,0.790123,0.0
1,1.0,0.307512,0.0,0.320988,1.000000,0.0


In [5]:
df = pd.merge(df, temp, how='left', on=['Sex'])
df = df.drop(['Sex'] , axis=1)
df.head()

,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Mean,Age_Mode,Age_Median,Age_Max,Age_Min
0,1.0,0.121348,0.283951,0.125,0.0,0.769118,0.014151,0.000000,1.000000,0.307512,0.0,0.320988,1.000000,0.0
1,0.0,0.213483,0.481481,0.125,0.0,0.876471,0.139136,0.557823,0.333333,0.296729,0.0,0.308642,0.790123,0.0
2,1.0,0.396629,0.333333,0.000,0.0,0.983824,0.015469,0.000000,1.000000,0.296729,0.0,0.308642,0.790123,0.0
3,0.0,0.305618,0.444444,0.125,0.0,0.072059,0.103644,0.380952,1.000000,0.296729,0.0,0.308642,0.790123,0.0
4,1.0,0.016854,0.444444,0.000,0.0,0.694118,0.015713,0.000000,1.000000,0.307512,0.0,0.320988,1.000000,0.0


In [6]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

14 Numeric Features : ['Pclass', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Age_Mean', 'Age_Mode', 'Age_Median', 'Age_Max', 'Age_Min']



,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Mean,Age_Mode,Age_Median,Age_Max,Age_Min
0,1.0,0.121348,0.283951,0.125,0.0,0.769118,0.014151,0.000000,1.000000,0.307512,0.0,0.320988,1.000000,0.0
1,0.0,0.213483,0.481481,0.125,0.0,0.876471,0.139136,0.557823,0.333333,0.296729,0.0,0.308642,0.790123,0.0
2,1.0,0.396629,0.333333,0.000,0.0,0.983824,0.015469,0.000000,1.000000,0.296729,0.0,0.308642,0.790123,0.0
3,0.0,0.305618,0.444444,0.125,0.0,0.072059,0.103644,0.380952,1.000000,0.296729,0.0,0.308642,0.790123,0.0
4,1.0,0.016854,0.444444,0.000,0.0,0.694118,0.015713,0.000000,1.000000,0.307512,0.0,0.320988,1.000000,0.0


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [7]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""

df_minus = df.drop(['Age_Mean', 'Age_Mode', 'Age_Median', 'Age_Max', 'Age_Min'] , axis=1)

# 原始特徵 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(df_minus.astype('float'))
estimator = LogisticRegression(solver='lbfgs')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6982329516378364

In [8]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df.astype('float'))
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.8002301237646673